In [ ]:
mp3_file="input/some-voice-person.mp3"

## Start model
Run this only once. It is slow.

In [46]:
import soundfile as sf
import numpy as np
from voxcpm import VoxCPM

model = VoxCPM.from_pretrained("openbmb/VoxCPM-0.5B")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

voxcpm_model_path: /home/finn/.cache/huggingface/hub/models--openbmb--VoxCPM-0.5B/snapshots/f67d35a3848e0bec0fdb8c33e6fc92cf293ee72f, zipenhancer_model_path: iic/speech_zipenhancer_ans_multiloss_16k_base, enable_denoiser: True


2025-09-25 20:55:43,124 - modelscope - INFO - initiate model from /home/finn/.cache/modelscope/hub/models/iic/speech_zipenhancer_ans_multiloss_16k_base
2025-09-25 20:55:43,125 - modelscope - INFO - initiate model from location /home/finn/.cache/modelscope/hub/models/iic/speech_zipenhancer_ans_multiloss_16k_base.
2025-09-25 20:55:43,125 - modelscope - INFO - initialize model from /home/finn/.cache/modelscope/hub/models/iic/speech_zipenhancer_ans_multiloss_16k_base
2025-09-25 20:55:43,205 - modelscope - WARNING - No preprocessor field found in cfg.
2025-09-25 20:55:43,206 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2025-09-25 20:55:43,206 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/finn/.cache/modelscope/hub/models/iic/speech_zipenhancer_ans_multiloss_16k_base'}. trying to build by task and model information.
2025-09-25 20:55:43,206 - model

Warm up VoxCPMModel...


100%|██████████| 10/10 [00:01<00:00,  5.20it/s]


## Start SenseVoiceSmall model, Transcribe

In [47]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

model_dir = "iic/SenseVoiceSmall"
sense_model = AutoModel(
    model=model_dir,
    disable_update=True, # Fjern hvis du vil oppdatere
    vad_model="fsmn-vad",
    vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
)

funasr version: 1.2.7.


## Transcripbe mp3->text

In [64]:
# en
res = sense_model.generate(
    input=f"{mp3_file}",
    cache={},
    language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
transcribed_text = rich_transcription_postprocess(res[0]["text"])
print(transcribed_text)

rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 41.30it/s]
rtf_avg: 0.002, time_speech:  14.720, time_escape: 0.027: 100%|██████████| 1/1 [00:00<00:00, 26.96it/s]

And that might scare some people. It scares me. I've had to kill projects use using event sourcing at least one time in my career because they basically did what I talked about earlier, right with the what beginners do.


## mp3 to wav

In [67]:
from pydub import AudioSegment
import os
wav_file = f"{mp3_file}.wav"
if not os.path.exists(mp3_file):
    print(f"Error: Input file '{mp3_file}' not found.")
else:
    try:
        sound = AudioSegment.from_mp3(mp3_file)
        sound.export(wav_file, format="wav")
        print(f"Successfully converted '{mp3_file}' to '{wav_file}'")
    except Exception as e:
        print(f"An error occurred during conversion: {e}")

Successfully converted 'input/august-datomic.mp3' to 'input/august-datomic.mp3.wav'


## wav+transcribe -> output

In [ ]:
wavOut = model.generate(
    text="Tina is giving birth soon. I hope I am the father. The mail man around here is very handsome.",
    prompt_wav_path=f"{wav_file}",      # optional: path to a prompt speech for voice cloning
    prompt_text=f"{transcribed_text}",          # optional: reference text
    cfg_value=2.0,             # LM guidance on LocDiT, higher for better adherence to the prompt, but maybe worse
    inference_timesteps=10,   # LocDiT inference timesteps, higher for better result, lower for fast speed
    normalize=True,           # enable external TN tool
    denoise=True,             # enable external Denoise tool
    retry_badcase=True,        # enable retrying mode for some bad cases (unstoppable)
    retry_badcase_max_times=3,  # maximum retrying times
    retry_badcase_ratio_threshold=6.0, # maximum length restriction for bad case detection (simple but effective), it could be adjusted for slow pace speech
)

sf.write("output/output.wav", wavOut, 16000)
print("saved: output.wav")

inputs:(1, 235512)
decode_do_segement
padding: 12488
inputs after padding:(1, 248000)
current_idx: 240000 100.00%


 81%|████████  | 125/154 [00:03<00:00, 35.48it/s]

saved: output.wav


## Streaming .wav
Havent tried this and dont know what its useful for.

In [ ]:
# Streaming
chunks = []
for chunk in model.generate_streaming(
    text = "Streaming text to speech is easy with VoxCPM!",
    # supports same args as above
):
    chunks.append(chunk)
wav = np.concatenate(chunks)

sf.write("output/streaming.wav", wav, 16000)
print("saved: output_streaming.wav")